<a href="https://colab.research.google.com/github/kjiky7777/MBA-Project/blob/main/Modeling_(NLP%2C_Feature_Reduction_X).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from yellowbrick.classifier import ROCAUC
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
import time

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 데이터 불러오기


X =  pd.read_csv('/content/drive/My Drive/finbert/FRED.csv')

name = pd.read_csv('/content/drive/My Drive/finbert/FRED_appendix.csv')

Y = pd.read_csv('/content/drive/My Drive/finbert/S&P500.csv')

In [ ]:
print(X.shape, Y.shape)

(267, 128) (267, 8)


In [ ]:
# 전처리 과정

del X['sasdate']

Y = Y['Result']

X.rename(columns={'IPB51222S':'IPB51222s'},inplace=True)

X = X.fillna(0)


# 변수 선택

feature_name = name['fred']

X = X.loc[:,feature_name]


In [ ]:
X

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,IPFPNSS,IPFINAL,IPDCONGD,IPNCONGD,IPBUSEQ,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,10861.073,9498.6,76.746,1129614,268044,99.3409,96.5258,98.4850,102.5384,83.8829,...,72.498,71.832,15.01,17.11,14.13,112.0,132467.09,421056.34,1147.6935,24.8430
1,10906.015,9541.8,77.468,1109286,272020,99.8416,97.1313,98.1898,103.7785,84.5120,...,73.025,72.046,15.06,17.20,14.14,111.3,136610.16,422437.48,1140.6992,25.4560
2,10936.525,9573.1,77.850,1122401,275192,100.0390,97.2144,97.8025,103.3591,85.6665,...,73.959,72.174,15.10,17.28,14.17,107.1,136745.97,423323.75,1153.1385,24.7678
3,10996.136,9620.5,77.827,1131729,271046,100.8609,98.0194,99.0959,104.0481,86.6401,...,73.660,72.207,15.17,17.39,14.23,109.2,136863.39,427912.64,1160.1010,29.8331
4,11035.828,9628.6,78.118,1125723,271394,101.0331,98.3635,98.7398,104.4840,87.4207,...,73.531,72.346,15.17,17.38,14.22,110.7,140410.20,432485.03,1164.3110,29.4290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,17801.835,14463.1,125.640,1572476,644741,101.2536,101.6843,104.7551,99.6725,95.9763,...,105.852,126.278,26.99,31.04,24.24,67.4,369983.50,754662.69,5607.1160,19.1586
263,17791.488,14465.3,123.868,1562935,634393,100.8807,101.3610,103.9402,99.3044,95.9503,...,106.351,126.834,27.17,31.25,24.38,70.6,370554.92,753781.25,5678.9640,21.2985
264,17711.363,14444.5,125.781,1581814,651557,101.7294,102.2818,105.6880,100.5197,95.5580,...,107.154,127.248,27.31,31.44,24.51,67.2,370286.86,751635.23,5777.3631,22.9143
265,17727.726,14485.8,125.878,1570284,662321,102.9275,103.3749,103.8407,101.8127,97.3472,...,109.075,127.566,27.42,31.60,24.56,62.8,374193.90,753730.65,5818.5663,26.1429


## Logisitc Regression

In [ ]:
np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter' : [1000,20000]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

grid_search.fit(X_train, Y_train)



In [ ]:
print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'C': 0.01, 'max_iter': 1000}


In [ ]:
# 모델 구축

model = LogisticRegression(C = 10 , max_iter=30000, random_state=777)

In [ ]:
X.iloc[252:268,:]

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,IPFPNSS,IPFINAL,IPDCONGD,IPNCONGD,IPBUSEQ,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
252,19103.244,14031.9,119.498,1579345,572822,99.0915,100.0551,107.7847,98.8359,93.9628,...,99.980,121.742,25.88,29.71,23.27,79.0,355240.05,738832.82,4774.7422,25.5982
253,17678.432,14028.9,117.905,1531244,562686,96.9818,98.0210,100.1276,98.2046,91.2959,...,100.456,121.992,25.94,29.82,23.39,76.8,355291.71,736048.95,4864.5907,22.5607
254,21267.506,14124.2,123.299,1591781,625731,98.9656,99.7706,102.1429,99.2276,94.0118,...,101.233,122.594,26.00,29.84,23.40,84.9,358586.15,743388.12,4976.8915,21.4680
255,18328.957,14199.2,123.866,1585058,626430,98.3697,99.0437,99.2309,99.4554,92.3306,...,101.516,123.120,26.16,30.12,23.48,88.3,362666.77,751213.18,5077.8221,17.1684
256,17875.090,14242.7,123.253,1551428,620075,98.8980,99.7605,101.5654,99.5969,93.7686,...,101.657,123.565,26.31,30.27,23.64,82.9,365154.60,754248.23,5182.9647,19.1336
257,17827.219,14277.9,123.981,1559335,626999,98.9005,99.8743,100.0380,99.7533,93.2133,...,102.223,124.093,26.44,30.43,23.77,85.5,366098.79,756290.99,5254.4912,16.9236
258,17985.182,14345.8,123.593,1558848,620648,100.2765,101.4638,104.5418,99.7102,95.9640,...,102.803,124.557,26.58,30.61,23.89,81.2,366126.54,756089.86,5299.2169,17.5754
259,17986.302,14340.7,124.496,1556107,624738,100.0586,100.9565,102.4213,99.8629,95.0837,...,103.225,124.913,26.73,30.76,24.03,70.3,366753.45,756588.86,5354.4097,17.0618
260,17755.780,14372.5,124.826,1563118,630905,99.4979,100.0914,99.1582,99.5798,94.2317,...,103.808,125.243,26.81,30.76,24.10,72.8,370236.20,758672.57,5418.2216,19.7806
261,17785.166,14440.2,125.740,1570449,640899,100.2852,100.8200,103.9343,98.7532,95.2649,...,104.958,125.652,26.88,30.95,24.17,71.7,369540.39,755823.28,5529.1918,17.7838


In [ ]:
pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :]
  Y_train = Y.iloc[0:252+i]

  model.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:]
  pred.append( int(model.predict(X_test.array.reshape(1,-1))) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/b

In [ ]:
print(pred)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0]


In [ ]:
Y_test = Y.iloc[252:269]

len(Y_test)

15

In [ ]:
import numpy as np
np.array(pred)

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0])

In [ ]:
np.array(Y_test)

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1])

In [ ]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

(2, 3, 4, 6)

In [ ]:
# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)


Accuracy :  0.5333333333333333
Precision :  0.6666666666666666
Recall :  0.6
TPR :  0.75
FPR :  0.5


In [ ]:
print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

roc auc value 0.5


In [ ]:
model.score(X_train,Y_train)

NotFittedError: ignored

In [ ]:
# 변수 중요도

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

model_fit = model.fit(X_train, Y_train)
cross_val_score(model_fit, X_train, Y_train, cv=5)


feature_importance_lr = pd.DataFrame(zip(X.columns.values, model_fit.coef_.ravel()))
feature_importance_lr.columns = ['feature', 'coef']
feature_importance_lr.sort_values("coef", ascending=False, inplace=True)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

In [ ]:
feature_importance_lr

,feature,coef
38,USFIRE,0.002859
52,M2REAL,0.002839
51,M2SL,0.002409
21,UEMPLT5,0.000871
28,USGOOD,0.000837
...,...,...
1,W875RX1,-0.001187
97,INVEST,-0.001250
35,USTPU,-0.002193
43,HOUST,-0.002711


In [ ]:
feature_importance_pos = feature_importance_lr[0:5]

# 지수 상승할 확률 Rate배 증가 
feature_importance_pos['Rate'] = np.exp(feature_importance_pos['coef'])
feature_importance_pos

del feature_importance_pos['coef']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
feature_importance_neg = feature_importance_lr[-5:]

# 지수 상승할 확률 Rate배 감소
feature_importance_neg['Rate'] = 1 - np.exp(feature_importance_neg['coef'])
feature_importance_neg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,feature,coef,Rate
34,SRVPRD,-0.000496,0.000496
35,USTPU,-0.000570,0.000570
43,HOUST,-0.000593,0.000593
50,M1SL,-0.000696,0.000696
39,USGOVT,-0.001537,0.001535


In [ ]:
# Tree 사용

# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"


# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(777)


In [ ]:
# RandomForest

from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Grid Search

np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators' : [100, 200,300],
           'max_depth' : [6, 8, 10, 12, 16, 20],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8,16,20] # 작을수록 overfitting
            
            }

rf_clf = RandomForestClassifier(random_state = 777, n_jobs = -1)
grid_search = GridSearchCV(rf_clf, param_grid, cv=5 , refit=True, n_jobs=-1)

grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=777),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'max_depth': 6, 'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 100}


In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    min_samples_leaf=18,
    min_samples_split=16,
    random_state=777,
    n_jobs=-1
)

In [ ]:
pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :]
  Y_train = Y.iloc[0:252+i]

  rf_clf.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:]
  pred.append( int(rf_clf.predict(X_test.array.reshape(1,-1))) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with

In [ ]:
pred

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [ ]:
Y_test= Y.iloc[252:267]
len(Y_test)

15

In [ ]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

(0, 5, 1, 9)

In [ ]:
# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)

Accuracy :  0.6
Precision :  0.6428571428571429
Recall :  0.9
TPR :  1.125
FPR :  0.0


In [ ]:
print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

roc auc value 0.45


In [ ]:
rf_clf.score(X_train,Y_train)

0.7142857142857143

# **과적합 현상 발생, 변수 축소**




In [ ]:
# 데이터 불러오기


X =  pd.read_csv('/content/drive/My Drive/finbert/FRED.csv')

Y = pd.read_csv('/content/drive/My Drive/finbert/S&P500.csv')

In [ ]:
# 변수 축소


feature_re =  pd.read_csv('/content/drive/My Drive/finbert/new_feature.csv')

del X['sasdate']

Y = Y['Result']

X.rename(columns={'IPB51222S':'IPB51222s'},inplace=True)

X = X.fillna(0)


# 변수 선택

new_feature_name = feature_re['fred']

X = X.loc[:,new_feature_name]

del X['Positive']
del X['Negative']
del X['Neutral']



## Logistic Regression

In [ ]:
np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter' : [1000,10000,20000]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

grid_search.fit(X_train, Y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'max_iter': [1000, 20000]})

In [ ]:
print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'learning_rate': 0.001, 'n_estimators': 200}


In [ ]:
logit_clf = LogisticRegression(
    random_state = 777,
    C = 0.1,
    max_iter = 20000
)


pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :]
  Y_train = Y.iloc[0:252+i]

  logit_clf.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:]
  pred.append( int(logit_clf.predict(X_test.array.reshape(1,-1))) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X 

In [ ]:
print(pred)

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# Confusion Matrix

Y_test= Y.iloc[252:267]


from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)

Accuracy :  0.6666666666666666
Precision :  0.6923076923076923
Recall :  0.9
TPR :  1.125
FPR :  0.25


In [ ]:
print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

logit_clf.score(X_train,Y_train)

roc auc value 0.55


0.6729323308270677

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Grid Search



np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators' : [100, 200,300],
           'max_depth' : [6, 8, 10, 12, 16, 20],
           'min_samples_leaf' : [5, 7, 10,12],
           'min_samples_split' : [3,5,7,10,12] # 작을수록 overfitting
            }

rf_clf = RandomForestClassifier(random_state = 777, n_jobs = -1)
grid_search = GridSearchCV(rf_clf, param_grid, cv=5 , refit=True, n_jobs=-1)

grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=777),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20],
                         'min_samples_leaf': [5, 7, 10, 12],
                         'min_samples_split': [3, 5, 7, 10, 12],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 3, 'n_estimators': 200}


In [ ]:
# RandomForest 재적합
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    min_samples_leaf=12,
    min_samples_split=3,
    random_state=777,
    n_jobs=-1
)

pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :]
  Y_train = Y.iloc[0:252+i]

  rf_clf.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:]
  pred.append( int(rf_clf.predict(X_test.array.reshape(1,-1))) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with

In [ ]:
print(pred)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]


In [ ]:
print(Y_test)

252    0
253    1
254    1
255    1
256    1
257    1
258    1
259    1
260    0
261    1
262    0
263    1
264    0
265    0
266    1
Name: Result, dtype: int64


In [ ]:
# Confusion Matrix

Y_test= Y.iloc[252:267]


from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

(1, 4, 1, 9)

In [ ]:
# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)

Accuracy :  0.6666666666666666
Precision :  0.6923076923076923
Recall :  0.9
TPR :  1.125
FPR :  0.25


In [ ]:
print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

rf_clf.score(X_train,Y_train)

roc auc value 0.55


0.7481203007518797

## ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Grid Search

np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators' : [100, 200,300],
               'learning_rate' : [0.001, 0.01, 0.1, 0.5, 1]
            }

ada_clf = AdaBoostClassifier(random_state = 777)
grid_search = GridSearchCV(ada_clf, param_grid)
grid_search.fit(X_train, Y_train)

GridSearchCV(estimator=AdaBoostClassifier(random_state=777),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'learning_rate': 0.001, 'n_estimators': 100}


In [ ]:
#ADABoost 적합


ada_clf = AdaBoostClassifier(
    learning_rate = 0.001,
    n_estimators=100,
    random_state=777
)

pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :]
  Y_train = Y.iloc[0:252+i]

  ada_clf.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:]
  pred.append( int(ada_clf.predict(X_test.array.reshape(1,-1))) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  "X 

In [ ]:
print(pred)

[0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]


In [ ]:
# Confusion Matrix

Y_test= Y.iloc[252:267]


from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

(5, 0, 5, 5)

In [ ]:
# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)

Accuracy :  0.6666666666666666
Precision :  1.0
Recall :  0.5
TPR :  0.625
FPR :  1.25


In [ ]:
print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

print('training score : ',ada_clf.score(X_train,Y_train))

roc auc value 0.75
training score :  0.6691729323308271


## XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Grid Search

np.random.seed(777)

#grid_search 이용
X_train = X.iloc[0:266,:]
Y_train = Y.iloc[0:266]

from sklearn.model_selection import GridSearchCV

param_grid = {  
           'n_estimators' : [100,200,300,500,700,1000],
           'max_depth' : [3,4,5,6, 8, 10, 12],
           'learning_rate' : [0.001,0.003,0.01,0.03,0.1,0.3],
            'objective' : ['binary:logistic'],
            'min_child_weight' : [1,3,5,7,10],
            'gamma' : [0,1] 
            
            }

xgb_clf = XGBClassifier(random_state = 777)
grid_search = GridSearchCV(xgb_clf, param_grid)
grid_search.fit(X_train, Y_train)

print('최적의 parameters : ', grid_search.best_params_)

최적의 parameters :  {'gamma': 0, 'learning_rate': 0.001, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 300, 'objective': 'binary:logistic'}


In [ ]:
xgb_clf = XGBClassifier(
    gamma = 0,
    learning_rate = 0.001,
    max_depth = 3,
    min_child_weight = 10,
    n_estimators = 300,
    objective = 'binary:logistic',
    random_state = 777
)


pred = []
for i in range(0,15) :

  X_train = X.iloc[0:252+i, :].values
  Y_train = Y.iloc[0:252+i].values

  xgb_clf.fit(X_train,Y_train)

  X_test = X.iloc[252+i,:].to_numpy()
  pred.append( int(xgb_clf.predict(np.array(X_test).reshape((1,-1)))))

In [ ]:
pred

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [ ]:
# Confusion Matrix

Y_test= Y.iloc[252:267]


from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(Y_test), np.array(pred))

tn, fp, fn, tp = confusion_matrix(np.array(Y_test), np.array(pred)).ravel()
(tn, fp, fn, tp)

# Accuracy
accuracy =  (tp+tn)/(tp+tn+fp+fn)
print('Accuracy : ', accuracy)

# Precision
precision = tp/(tp+fp)
print('Precision : ', precision)

# Recall
recall = tp/(tp+fn)
print('Recall : ', recall)

# TPR
tpr = tp / 8
print('TPR : ', tpr)

# FPR
fpr = tn / 4
print('FPR : ' , fpr)

print('roc auc value {}'.format(roc_auc_score(Y_test,pred)))

print('training score : ',xgb_clf.score(X_train,Y_train))

Accuracy :  0.6666666666666666
Precision :  0.8571428571428571
Recall :  0.6
TPR :  0.75
FPR :  1.0
roc auc value 0.7000000000000002
training score :  0.7368421052631579


In [ ]:
xgb_clf.fit(X_train, Y_train)
 
visualizer = ROCAUC(xgb_clf, classes=[0, 1], micro=False, macro=False, per_class=False)
visualizer.fit(X_train, Y_train)
visualizer.score(X_train, Y_train)
visualizer.show()

IndexError: ignored